In [10]:
### https://www.youtube.com/watch?v=xdq6Gz33khQ

In [1]:
import requests
import base64
import datetime
from urllib.parse import urlencode


In [23]:
client_id='4b41bb194b254d86ab6f5e7ab9389666'
client_secret = '0ac4982003584fb7959a346c9b3f2a59'

In [40]:
playlists = {'happy':'', 
            'sad':'',
            'disgust':'5GX0cpDTiyqK7VNZpA55Wc',
            'anger':''}

In [25]:
class SpotifyAPI(object):
    access_token = None
    access_token_expires = datetime.datetime.now()
    access_token_did_status = True     # True if not expired and False if expiried
    client_id = None 
    client_secret = None 
    token_url= "https://accounts.spotify.com/api/token"
    
    def __init__(self, client_id, client_secret, *args, **kwargs): 
        super().__init__(*args, **kwargs)
        self.client_id = client_id
        self.client_secret = client_secret
    
    def get_client_creditials(self): 
        """
            reuturns a base 64 encoded string././
        """
        self.client_id = client_id
        self.client_secret = client_secret
        
        if client_secret == None or client_id == None : 
            raise Exception("You must set client id and client sceret") 
        client_creds= f'{client_id}:{client_secret}'
        client_creds_b64=base64.b64encode(client_creds.encode())
        
        return client_creds_b64
    
    def get_token_headers(self): 
        client_creds_b64 = self.get_client_creditials()
        return { 
                "Authorization" : f'Basic {client_creds_b64.decode()}'  # <base64 encoded client_id:client_secret > 
                }
    def get_token_data(self):
        return {  
                'grant_type' : 'client_credentials'
                }
    def get_resource_header(self):
        access_token=self.get_access_token() 
        headers={
            "Authorization" : f"Bearer {access_token}"   
            }
        return headers
    
    def get_resource(self,lookup_id,resource_type='albums',version='v1'):
        endpoint=f'https://api.spotify.com/{version}/{resource_type}/{lookup_id}'
        headers=self.get_resource_header()
        r=requests.get(endpoint,headers=headers)
        if r.status_code not in range(200,299):         # returns a boolean value. checking http stauts reponse 
            return {'No results'}
        return r.json()
    
#     def get_album(self,_id):
#         base_url='https://api.spotify.com/'
#         endpoint=f'{base_url}/v1/album/{_id}'
#         pass

    def get_album(self,_id): 
        return self.get_resource(_id,resource_type='albums')
    def get_artist(self,_id):
            return self.get_resource(_id,resource_type='artists')
        
#     def get_atrist(seldf,_id):
#         base_url='https://api.spotify.com/'
#         endpoint=f'{base_url}/v1/artist/{_id}'
#         pass
    
    def get_access_token(self):
#         auth_done=self.perform_auth()
#         if not auth_done:
#             raise Exception("Authentication Failed")
        token=self.access_token
        expiry_time=self.access_token_expires
        current_time=datetime.datetime.now() 
        if expiry_time < current_time:
            self.perform_auth()
            return self.get_access_token()
        elif token == None:
            self.perform_auth()
            return self.get_access_token()
        return token
        
    def perform_auth(self):
        token_url=self.token_url
        token_data=self.get_token_data()
        token_headers=self.get_token_headers() 
        
        r=requests.post(token_url,data=token_data, headers=token_headers) 
        if r.status_code not in range(200,299):         # returns a boolean value. checking http stauts reponse 
            raise Exception("Could not authenticate client")
        
        data=r.json()
        current_datetime=datetime.datetime.now() 
        access_token=data['access_token']
        expires_in=data['expires_in'] # seconds
        expires_time_left = current_datetime + datetime.timedelta(seconds=expires_in)
        self.access_token=access_token
        self.access_token_expires = expires_time_left
        self.access_token_expires_status = expires_time_left < current_datetime
        return True
    
    def base_search(self,query_params):
        access_token=self.get_access_token() 
        headers=self.get_resource_header()
        endpoint='https://api.spotify.com/v1/search'
        lookup_url=f'{endpoint}?{query_params}'
        r=requests.get(lookup_url ,headers=headers)
        if r.status_code not in range(200,299):         # returns a boolean value. checking http stauts reponse 
            return {'No results'}
        return r.json()
    
    def search(self, query=None,search_type='artists'):
        if query == None:
            raise Exception("A query is required")
        if isinstance(query,dict):
            query=" ".join([f'{k}:{v}' for k,v in query.items()])
        query_params=urlencode({"q": query, "type" : search_type.lower()})
        return self.base_search(query_params)
    
    def get_playlist(self,_id):
        access_token = self.get_access_token()
        playlist_url = f'https://api.spotify.com/v1/playlists/{_id}'
        headers={
                    "Authorization" : f"Bearer {access_token}"   
                    }
        response=requests.get(playlist_url, headers=headers)
        return response #self.get_resource(_id,resource_type='playlist')



In [26]:
spotify = SpotifyAPI(client_id, client_secret)

In [5]:
xx=spotify.search("yellow submarine", search_type="Track")

In [41]:
playlists['disgust']

'5GX0cpDTiyqK7VNZpA55Wc'

In [42]:
spotify.get_playlist(playlists['disgust']).content

b'{\n  "collaborative" : false,\n  "description" : "",\n  "external_urls" : {\n    "spotify" : "https://open.spotify.com/playlist/5GX0cpDTiyqK7VNZpA55Wc"\n  },\n  "followers" : {\n    "href" : null,\n    "total" : 0\n  },\n  "href" : "https://api.spotify.com/v1/playlists/5GX0cpDTiyqK7VNZpA55Wc",\n  "id" : "5GX0cpDTiyqK7VNZpA55Wc",\n  "images" : [ {\n    "height" : 640,\n    "url" : "https://i.scdn.co/image/ab67616d0000b2730b345678ef9735630f459341",\n    "width" : 640\n  } ],\n  "name" : "SER_disgust",\n  "owner" : {\n    "display_name" : "Ia",\n    "external_urls" : {\n      "spotify" : "https://open.spotify.com/user/4rjaqt6oilgawvahbml31jfd7"\n    },\n    "href" : "https://api.spotify.com/v1/users/4rjaqt6oilgawvahbml31jfd7",\n    "id" : "4rjaqt6oilgawvahbml31jfd7",\n    "type" : "user",\n    "uri" : "spotify:user:4rjaqt6oilgawvahbml31jfd7"\n  },\n  "primary_color" : null,\n  "public" : false,\n  "snapshot_id" : "NCxhZGFkZTkzZmVmM2RhOTZhNmM5ZTVkZmM0ZjQyOWIyNTQ2MmQ4ZDI2",\n  "tracks" : 

NameError: name 'response' is not defined

In [ ]:
def get_playlist(self,_id):
        access_token = self.get_access_token()
        playlist_url = f'https://api.spotify.com/v1/playlists/{_id}'
        headers={
                    "Authorization" : f"Bearer {access_token}"   
                    }

        response=requests.get(playlist_url, headers=headers)

        return response #self.get_resource(_id,resource_type='playlist')


In [6]:
spotify.get_artist("3tJoFztHeIJkJWMrx0td2f")

{'external_urls': {'spotify': 'https://open.spotify.com/artist/3tJoFztHeIJkJWMrx0td2f'},
 'followers': {'href': None, 'total': 2301830},
 'genres': ['memphis hip hop',
  'rap',
  'southern hip hop',
  'tennessee hip hop',
  'trap'],
 'href': 'https://api.spotify.com/v1/artists/3tJoFztHeIJkJWMrx0td2f',
 'id': '3tJoFztHeIJkJWMrx0td2f',
 'images': [{'height': 640,
   'url': 'https://i.scdn.co/image/ab6761610000e5eb9d566d7d5ca673f9af94f4ce',
   'width': 640},
  {'height': 320,
   'url': 'https://i.scdn.co/image/ab676161000051749d566d7d5ca673f9af94f4ce',
   'width': 320},
  {'height': 160,
   'url': 'https://i.scdn.co/image/ab6761610000f1789d566d7d5ca673f9af94f4ce',
   'width': 160}],
 'name': 'Moneybagg Yo',
 'popularity': 81,
 'type': 'artist',
 'uri': 'spotify:artist:3tJoFztHeIJkJWMrx0td2f'}

In [7]:
spotify.get_album("7vEJAtP3KgKSpOHVgwm3Eh")

{'album_type': 'album',
 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/3WrFJ7ztbogyGnTHbHJFl2'},
   'href': 'https://api.spotify.com/v1/artists/3WrFJ7ztbogyGnTHbHJFl2',
   'id': '3WrFJ7ztbogyGnTHbHJFl2',
   'name': 'The Beatles',
   'type': 'artist',
   'uri': 'spotify:artist:3WrFJ7ztbogyGnTHbHJFl2'}],
 'available_markets': ['AD',
  'AE',
  'AG',
  'AL',
  'AM',
  'AO',
  'AR',
  'AT',
  'AU',
  'AZ',
  'BA',
  'BB',
  'BD',
  'BE',
  'BF',
  'BG',
  'BH',
  'BI',
  'BJ',
  'BN',
  'BO',
  'BR',
  'BS',
  'BT',
  'BW',
  'BY',
  'BZ',
  'CA',
  'CH',
  'CI',
  'CL',
  'CM',
  'CO',
  'CR',
  'CV',
  'CW',
  'CY',
  'CZ',
  'DE',
  'DJ',
  'DK',
  'DM',
  'DO',
  'DZ',
  'EC',
  'EE',
  'EG',
  'ES',
  'FI',
  'FJ',
  'FM',
  'FR',
  'GA',
  'GB',
  'GD',
  'GE',
  'GH',
  'GM',
  'GN',
  'GQ',
  'GR',
  'GT',
  'GW',
  'GY',
  'HK',
  'HN',
  'HR',
  'HT',
  'HU',
  'ID',
  'IE',
  'IL',
  'IN',
  'IS',
  'IT',
  'JM',
  'JO',
  'JP',
  'KE',
  'KG',
  'KH'